# 01. 관측성 (Observability)

## 학습 목표

이 노트북에서는 Istio의 **관측성(Observability)** 기능을 학습합니다.

**핵심 개념:**
- 분산 추적 (Distributed Tracing)
- 메트릭 수집 (Metrics)
- 자동 텔레메트리 (Automatic Telemetry)

**학습 포인트:**
1. Envoy Sidecar가 자동으로 트레이스 헤더 전파
2. 코드 수정 없이 분산 트레이싱 구현
3. Service Mesh의 핵심 가치: 관측성 자동화

---

## 사전 조건 체크

In [ ]:
# 환경 확인
!kubectl get pods -n istio-demo | grep -E "httpbin|sleep"

---

## 개념 설명: 분산 추적

### 왜 분산 추적이 필요한가?

마이크로서비스 아키텍처에서 하나의 요청은 여러 서비스를 거칩니다:

```
사용자 → API Gateway → 서비스 A → 서비스 B → 데이터베이스
```

**문제점:**
- 어디서 지연이 발생했는지 파악 어려움
- 에러 발생 시 원인 추적 복잡
- 서비스 간 호출 관계 파악 어려움

### Istio의 해결책

**Envoy Sidecar**가 자동으로:
1. 각 요청에 고유한 Trace ID 부여
2. 서비스 간 호출 시 헤더로 전파
3. Jaeger로 트레이스 데이터 전송

**애플리케이션 코드 수정이 필요 없습니다!**

---

## 시나리오 1: 트래픽 생성 및 분산 트레이싱

먼저 트래픽을 생성하여 Jaeger에서 추적할 수 있는 데이터를 만듭니다.

In [ ]:
# 트래픽 10회 생성
print("트래픽 10회 생성 중...")
for i in range(10):
    !kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://httpbin.istio-demo:8000/get > /dev/null
    print(".", end="", flush=True)
print(" 완료!")

### 트레이스 헤더 확인

Envoy가 자동으로 추가하는 분산 추적 헤더를 확인합니다.

**주요 헤더:**
- `X-B3-Traceid`: 전체 트레이스의 고유 ID
- `X-B3-Spanid`: 현재 스팬(단위 작업)의 ID
- `X-B3-Parentspanid`: 부모 스팬 ID
- `X-Request-Id`: Envoy가 생성한 요청 ID

In [ ]:
# httpbin의 /headers 엔드포인트로 트레이스 헤더 확인
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://httpbin.istio-demo:8000/headers | grep -E "X-B3|X-Request-Id"

**결과 해석:**
- 이 헤더들은 Envoy가 자동으로 추가한 것
- 애플리케이션(httpbin)은 헤더를 받아서 그대로 반환
- 멀티 서비스 호출 시 이 헤더들이 전파되어 추적 가능

---

## 시나리오 2: Jaeger 트레이스 수집 확인

Jaeger API를 통해 수집된 트레이스를 확인합니다.

In [ ]:
# Jaeger에 등록된 서비스 목록
import json

result = !kubectl exec -n observability deploy/jaeger -- wget -qO- "http://localhost:16686/api/services" 2>/dev/null
if result:
    try:
        data = json.loads(result[0])
        print("=== 등록된 서비스 ===")
        for svc in data.get('data', []):
            print(f"  - {svc}")
    except:
        print("JSON 파싱 실패. 원본:", result)

In [ ]:
# 최근 트레이스 조회
import json

result = !kubectl exec -n observability deploy/jaeger -- wget -qO- "http://localhost:16686/api/traces?service=httpbin.istio-demo&limit=3" 2>/dev/null
if result:
    try:
        data = json.loads(result[0])
        traces = data.get('data', [])
        print(f"=== 최근 트레이스 ({len(traces)}개) ===")
        for t in traces:
            spans = t.get('spans', [])
            trace_id = t.get('traceID', 'unknown')[:16]
            print(f"  TraceID: {trace_id}... | Spans: {len(spans)}개")
    except:
        print("트레이스 조회 실패 (트래픽 생성 후 다시 시도)")

---

## 시나리오 4: OpenTelemetry Instrumentation (demo-app)

Istio의 자동 추적 외에도 **OpenTelemetry**를 사용하면 애플리케이션 내부 작업까지 추적할 수 있습니다.

`demo-app`은 다음 instrumentation을 포함합니다:
- Flask 자동 instrumentation
- Redis 자동 instrumentation
- MongoDB 자동 instrumentation
- HTTP Client (requests) 자동 instrumentation

### 아키텍처

```
demo-app (Flask + OTEL)
├── Redis (자동 instrumentation)
├── MongoDB (자동 instrumentation)
└── httpbin (HTTP client instrumentation + Istio 전파)
```

In [ ]:
# demo-app 상태 확인
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://demo-app:8080/ | python3 -m json.tool

### 4.1 Redis 트레이싱

Redis 명령어(SET, GET, INCR)가 각각 별도의 span으로 캡처됩니다.

In [ ]:
# Redis 데모 호출
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://demo-app:8080/redis-demo | python3 -m json.tool

### 4.2 MongoDB 트레이싱

MongoDB 작업(INSERT, FIND, AGGREGATE)도 자동으로 span이 생성됩니다.

In [ ]:
# MongoDB 데모 호출
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://demo-app:8080/mongo-demo | python3 -m json.tool

**예상 트레이스 구조 (Jaeger에서 확인):**

```
demo-app: GET /mongo-demo
├── mongo-operations (custom span)
│   ├── mongo-insert (custom span)
│   │   └── pymongo.insert_one (auto span)
│   ├── mongo-find (custom span)
│   │   └── pymongo.find (auto span)
│   └── mongo-aggregate (custom span)
│       └── pymongo.aggregate (auto span)
```

### 4.3 Redis vs MongoDB 트레이싱 비교

| 항목 | Redis | MongoDB |
|------|-------|---------|
| 라이브러리 | `redis-py` | `pymongo` |
| OTEL Instrumentation | `opentelemetry-instrumentation-redis` | `opentelemetry-instrumentation-pymongo` |
| 캡처되는 정보 | 명령어, 키 | 컬렉션, 쿼리 필터 |
| Span 이름 예시 | `SET`, `GET`, `INCR` | `insert`, `find`, `aggregate` |

### 4.4 Full Demo (Redis + MongoDB + HTTP 통합)

`/full-demo` 엔드포인트는 모든 백엔드를 호출하여 하나의 트레이스에서 전체 흐름을 확인할 수 있습니다.

In [ ]:
# Full Demo 호출 (Redis + MongoDB + httpbin 모두 포함)
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://demo-app:8080/full-demo | python3 -m json.tool

**Full Demo 트레이스 구조:**

```
demo-app: GET /full-demo
├── full-demo-flow
│   ├── business-logic
│   ├── cache-check
│   │   └── redis.GET / redis.SET (auto spans)
│   ├── mongodb-store
│   │   └── pymongo.insert_one / pymongo.count_documents (auto spans)
│   ├── external-api-call
│   │   └── HTTP GET httpbin/get (+ Istio span 전파)
│   └── process-results
```

**Jaeger에서 확인할 포인트:**
1. Service: `demo-app` 선택
2. Operation: `GET /full-demo` 선택
3. 트레이스 클릭하여 상세 span 구조 확인
4. Redis, MongoDB, httpbin 각각의 지연 시간 비교

In [ ]:
# Jaeger에서 demo-app 트레이스 조회
import json

result = !kubectl exec -n observability deploy/jaeger -- wget -qO- "http://localhost:16686/api/traces?service=demo-app&limit=5" 2>/dev/null
if result:
    try:
        data = json.loads(result[0])
        traces = data.get('data', [])
        print(f"=== demo-app 최근 트레이스 ({len(traces)}개) ===")
        for t in traces:
            spans = t.get('spans', [])
            trace_id = t.get('traceID', 'unknown')[:16]
            operations = set(s.get('operationName', '') for s in spans)
            print(f"  TraceID: {trace_id}... | Spans: {len(spans)}개 | Operations: {', '.join(list(operations)[:3])}...")
    except:
        print("트레이스 조회 실패 (demo-app 호출 후 다시 시도)")

---

## 시나리오 3: Prometheus 메트릭 확인

Istio는 모든 서비스 간 통신에 대한 메트릭을 자동으로 수집합니다.

**주요 메트릭:**
- `istio_requests_total`: 총 요청 수
- `istio_request_duration_milliseconds`: 요청 지연 시간
- `istio_request_bytes_total`: 전송된 바이트 수

In [ ]:
# Prometheus에서 istio_requests_total 메트릭 조회
import json

result = !kubectl exec -n monitoring prometheus-prometheus-kube-prometheus-prometheus-0 -- wget -qO- 'http://localhost:9090/api/v1/query?query=istio_requests_total' 2>/dev/null
if result:
    try:
        data = json.loads(result[0])
        results = data.get('data', {}).get('result', [])
        print(f"=== istio_requests_total ({len(results)}개 시리즈) ===")
        for r in results[:5]:  # 상위 5개만 표시
            labels = r.get('metric', {})
            src = labels.get('source_workload', 'unknown')
            dst = labels.get('destination_workload', 'unknown')
            code = labels.get('response_code', '?')
            print(f"  {src} → {dst} (HTTP {code})")
    except:
        print("메트릭 조회 실패 (트래픽 생성 후 다시 시도)")

---

## UI 접속 방법

더 상세한 분석은 웹 UI를 통해 가능합니다.

### Jaeger UI (분산 트레이싱)

```bash
kubectl port-forward -n observability svc/jaeger 16686:16686
```
→ http://localhost:16686

### Grafana (대시보드)

```bash
kubectl port-forward -n monitoring svc/prometheus-grafana 3000:80
```
→ http://localhost:3000 (admin/admin)

### Prometheus (메트릭 쿼리)

```bash
kubectl port-forward -n monitoring svc/prometheus-kube-prometheus-prometheus 9090:9090
```
→ http://localhost:9090

### Kiali (서비스 메시 토폴로지)

```bash
kubectl port-forward -n istio-system svc/kiali 20001:20001
```
→ http://localhost:20001

---

## 관측성 테스트 완료!

### 핵심 정리

| 기능 | 도구 | 자동화 수준 |
|------|------|------------|
| 분산 추적 | Jaeger | 완전 자동 (헤더 전파) |
| 메트릭 수집 | Prometheus | 완전 자동 |
| 서비스 토폴로지 | Kiali | 완전 자동 |
| 대시보드 | Grafana | 설정 필요 |

**Istio의 관측성 장점:**
- 애플리케이션 코드 수정 불필요
- 모든 서비스에 일관된 메트릭/추적 적용
- 즉시 사용 가능한 대시보드

### 다음 단계

**[02-트래픽관리.ipynb](./02-트래픽관리.ipynb)** - 카나리 배포, 헤더 기반 라우팅 등 트래픽 관리 기능을 학습합니다.